In [22]:
# 此函数作用是对初始语料进行分词处理后，作为训练模型的语料
def cut_txt(old_file):
    import jieba
    global cut_file     # 分词之后保存的文件名
    cut_file = old_file + '_cut.txt'

    try:
        fi = open(old_file, 'r', encoding='utf-8')
    except BaseException as e:  # 因BaseException是所有错误的基类，用它可以获得所有错误类型
        print(Exception, ":", e)    # 追踪错误详细信息

    text = fi.read()  # 获取文本内容
    # jieba.load_userdict(“userDict.txt”) 加载用户自定义词典
    new_text = jieba.cut(text, cut_all=False)  # 精确模式
    str_out = ' '.join(new_text).replace('，', '').replace('。', '').replace('？', '').replace('！', '') \
        .replace('“', '').replace('”', '').replace('：', '').replace('…', '').replace('（', '').replace('）', '') \
        .replace('—', '').replace('《', '').replace('》', '').replace('、', '').replace('‘', '') \
        .replace('’', '')     # 去掉标点符号
    fo = open(cut_file, 'w', encoding='utf-8')
    fo.write(str_out)

In [23]:
def model_train(train_file_name, save_model_file):  # model_file_name为训练语料的路径,save_model为保存模型名
    from gensim.models import word2vec
    import gensim
    import logging
    # 模型训练，生成词向量
    logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
    sentences = word2vec.Text8Corpus(train_file_name)  # 加载语料
    model = gensim.models.Word2Vec(sentences, size=200,iter=20)  # 训练skip-gram模型; 默认window=5
    model.save(save_model_file)
    model.wv.save_word2vec_format(save_model_name + ".bin", binary=True)   # 以二进制类型保存模型以便重用

In [24]:
from gensim.models import word2vec
import os
import gensim

# if not os.path.exists(cut_file):    # 判断文件是否存在，参考：https://www.cnblogs.com/jhao/p/7243043.html
cut_txt('倚天屠龙记.txt')  # 须注意文件必须先另存为utf-8编码格式

save_model_name = '倚天屠龙记.model'
if not os.path.exists(save_model_name):     # 判断文件是否存在
    model_train(cut_file, save_model_name)
else:
    print('此训练模型已经存在，不用再次训练')

# 加载已训练好的模型
model_1 = word2vec.Word2Vec.load(save_model_name)
# 计算两个词的相似度/相关程度
y1 = model_1.similarity("赵敏", "韦一笑")
print(u"赵敏和韦一笑的相似度为：", y1)
print("-------------------------------\n")

# 计算某个词的相关词列表
y2 = model_1.most_similar("张三丰", topn=10)  # 10个最相关的
print(u"和张三丰最相关的词有：\n")
for item in y2:
    print(item[0], item[1])
print("-------------------------------\n")

2019-04-07 18:44:12,017 : INFO : collecting all words and their counts
2019-04-07 18:44:12,020 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2019-04-07 18:44:12,254 : INFO : collected 45115 word types from a corpus of 476041 raw words and 48 sentences
2019-04-07 18:44:12,254 : INFO : Loading a fresh vocabulary
2019-04-07 18:44:12,310 : INFO : effective_min_count=5 retains 10315 unique words (22% of original 45115, drops 34800)
2019-04-07 18:44:12,311 : INFO : effective_min_count=5 leaves 421126 word corpus (88% of original 476041, drops 54915)
2019-04-07 18:44:12,357 : INFO : deleting the raw counts dictionary of 45115 items
2019-04-07 18:44:12,361 : INFO : sample=0.001 downsamples 40 most-common words
2019-04-07 18:44:12,366 : INFO : downsampling leaves estimated 357491 word corpus (84.9% of prior 421126)
2019-04-07 18:44:12,406 : INFO : estimated required memory for 10315 words and 200 dimensions: 21661500 bytes
2019-04-07 18:44:12,407 : INFO : resetting 

2019-04-07 18:44:25,629 : INFO : worker thread finished; awaiting finish of 1 more threads
2019-04-07 18:44:25,632 : INFO : worker thread finished; awaiting finish of 0 more threads
2019-04-07 18:44:25,634 : INFO : EPOCH - 18 : training on 476041 raw words (357505 effective words) took 0.7s, 529209 effective words/s
2019-04-07 18:44:26,297 : INFO : worker thread finished; awaiting finish of 2 more threads
2019-04-07 18:44:26,301 : INFO : worker thread finished; awaiting finish of 1 more threads
2019-04-07 18:44:26,305 : INFO : worker thread finished; awaiting finish of 0 more threads
2019-04-07 18:44:26,306 : INFO : EPOCH - 19 : training on 476041 raw words (357551 effective words) took 0.7s, 535727 effective words/s
2019-04-07 18:44:26,968 : INFO : worker thread finished; awaiting finish of 2 more threads
2019-04-07 18:44:26,982 : INFO : worker thread finished; awaiting finish of 1 more threads
2019-04-07 18:44:26,986 : INFO : worker thread finished; awaiting finish of 0 more threads


赵敏和韦一笑的相似度为： 0.4054979
-------------------------------

和张三丰最相关的词有：

俞莲舟 0.6827965378761292
太师父 0.6131824254989624
宋远桥 0.577965259552002
觉远 0.5635546445846558
张真人 0.5622043609619141
师父 0.562029242515564
宋少侠 0.5554066896438599
俞岱岩 0.5527279376983643
赵小姐 0.5483282804489136
宋青书 0.5453828573226929
-------------------------------

